In [1]:
import numpy as np
np.set_printoptions(precision=3, suppress=True)

In [2]:
matrix_array = np.loadtxt('matrix_lu')
shape = matrix_array.shape

In [3]:
print(matrix_array)

[[  -7.    3.   -4.    7. -126.]
 [   8.   -1.   -7.    6.   29.]
 [   9.    9.    3.   -6.   27.]
 [  -7.   -9.   -8.   -5.   34.]]


## Метод LU-разложения

In [4]:
def decompose_column(matrix, column):
    l_matrix_column = np.zeros((shape[0], 1))
    for row in range (column + 1, shape[0]):
        l_matrix_value = (matrix[row, column] / matrix[column, column])
        l_matrix_column[row, 0] = l_matrix_value
        matrix[row, :] += matrix[column, :] * (-l_matrix_value)
        #log_step(column, matrix, row)
    return l_matrix_column

def decompose_column_lu(matrix, column):
    l_matrix_column = np.zeros(shape[0])
    for row in range (column + 1, shape[0]):
        l_matrix_value = (matrix[row, column] / matrix[column, column])
        l_matrix_column[row] = l_matrix_value
        matrix[row, column:] += matrix[column, column:] * (-l_matrix_value)
        #log_step(column, matrix, row)
    matrix[:, column] += l_matrix_column


def log_step(column, matrix, row):
    print(str(row) + " " + str(column))
    print(matrix)
    print('\n')


def get_matrix_with_non_zero_leading_element(matrix, column):
    for row in range (column, shape[0]):
        if matrix[row, column] != 0:
            matrix[[column, row]] = matrix[[row, column]]
            return column, row

def decompose_matrix_with_gauss(matrix):
    print("original memory")
    print(matrix.nbytes)
    b_column_to_swap = np.copy(matrix[:, shape[1] - 1].reshape(shape[0], 1))

    for column in range(0, shape[1] - 1):
        if matrix[column, column] == 0:
            row_swap_1, row_swap_2 = get_matrix_with_non_zero_leading_element(matrix, column)

            b_column_to_swap[[row_swap_1, row_swap_2]] = b_column_to_swap[[row_swap_2, row_swap_1]]

        decompose_column_lu(matrix, column)

    print("u memory")
    print(matrix.nbytes)
    return matrix, b_column_to_swap

def solve_lz_equation(matrix):
    answer = []
    for row in range (0, shape[0]):
        sum_of_polynomial = 0
        if row != 0:
            for column in range(0, row):
                sum_of_polynomial += matrix[row, column] * answer[column]

        x = matrix[row, shape[1] - 1] - sum_of_polynomial
        answer.append(x)

    return np.array(answer)

def solve_ux_equation(matrix):
    answer = []
    for row in range (shape[0] - 1, -1, -1):
        sum_of_polynomial = 0
        if row < shape[0]:
            for column in range (shape[0] - 1, row, -1):
                sum_of_polynomial +=  matrix[row, column] * answer[shape[0] - column - 1]

        x = (matrix[row, shape[1] - 1] - sum_of_polynomial) / matrix[row, row]
        answer.append(x)

    answer.reverse()
    return np.array(answer)


def solve_with_lu(matrix):
    u_matrix, swapped_b_column = decompose_matrix_with_gauss(matrix)
    log_matrix(u_matrix)
    #lz_matrix = np.hstack((l_matrix, swapped_b_column))
    u_matrix = np.delete(u_matrix, -1, axis=1)
    u_matrix = np.hstack((u_matrix, swapped_b_column))
    z_answer = solve_lz_equation(u_matrix)
    z_column = np.array(z_answer, dtype=float).reshape((shape[0], 1))
    u_matrix = np.delete(u_matrix, -1, axis=1)
    determinant = np.diagonal(u_matrix).prod()
    u_matrix = np.hstack((u_matrix, z_column))
    #print(lz_matrix)
    print("determinant")
    print(determinant)
    return solve_ux_equation(u_matrix)

def inverse_matrix(matrix):
    slau_matrix = np.delete(matrix, -1, axis=1)
    slau_matrix = np.hstack((slau_matrix, np.identity(shape[0])))

    for column in range(0, shape[1] - 1):
        if slau_matrix[column, column] == 0:
            get_matrix_with_non_zero_leading_element(slau_matrix, column)

        decompose_column(slau_matrix, column)
        slau_matrix[column, :] /= slau_matrix[column, column]

    for column in range(shape[1] - 1, -1, -1):
        decompose_column_backwards(slau_matrix, column - 1)

    slau_shape = slau_matrix.shape

    return slau_matrix[:, int(slau_shape[1] / 2) : int(slau_shape[1])]

def decompose_column_backwards(matrix, column):
    for row in range(column - 1, -1, -1):
        l_matrix_value = (matrix[row, column] / matrix[column, column])
        matrix[row, :] += matrix[column, :] * (-l_matrix_value)


def log_matrix(u_matrix):
    print("u matrix")
    print(u_matrix)
    print('\n')

In [5]:
matrix_array = np.loadtxt('matrix_lu')
print(solve_with_lu(matrix_array))
matrix_array = np.loadtxt('matrix_lu')
inversed = inverse_matrix(matrix_array)
print('inversed matrix')
print(inversed)

original memory
160
u memory
160
u matrix
[[  -7.       3.      -4.       7.    -126.   ]
 [  -1.143    2.429  -11.571   14.    -115.   ]
 [  -1.286    5.294   59.118  -71.118  473.824]
 [   1.      -4.941   -1.035  -16.418   82.09 ]]


determinant
16499.999999999996
[ 8. -9.  2. -5.]
inversed matrix
[[-0.055  0.055  0.006 -0.018]
 [ 0.086 -0.016  0.083  0.002]
 [-0.06  -0.05  -0.059 -0.073]
 [ 0.017  0.033 -0.063 -0.061]]


## Метод прогонки

In [6]:
matrix_array = np.loadtxt('matrix_sweep')
print(matrix_array)
shape = matrix_array.shape

[[  16.   -9.    0.    0.    0.  -27.]
 [   8.  -13.   -5.    0.    0.  -84.]
 [   0.   -3.  -21.    9.    0. -225.]
 [   0.    0.   -9.   16.   -5.  -89.]
 [   0.    0.    0.    1.   -9.   69.]]


In [7]:
def solve_sweep(matrix):
    if not check_tri_diagonal_dominance(matrix):
        return

    p = []
    q = []

    p.append(-c_i(matrix, 0) / b_i(matrix, 0))
    q.append(d_i(matrix, 0) / b_i(matrix, 0))

    for i in range(1, shape[0]):
        p.append(p_i(matrix, i, p))
        q.append(q_i(matrix, i, p ,q))

    answer = [0] * shape[0]
    answer[-1] = q[-1]

    for i in range(shape[0] - 2, -1, -1):
        answer[i] = x_i(i, p, q, answer)

    print("p coefficients")
    print(p)
    print("q coefficients")
    print(q)
    return np.array(answer)

def check_tri_diagonal_dominance(matrix):
    for i in range(shape[0]):
        if abs(b_i(matrix, i)) < (abs(a_i(matrix, i)) + abs(c_i(matrix, i))):
            return False
    return True

def p_i(matrix, i, p):
    return -c_i(matrix, i) / (b_i(matrix, i) + a_i(matrix, i) * p[i - 1])

def q_i(matrix, i, p ,q):
    return (d_i(matrix, i) - a_i(matrix, i) * q[i - 1]) / (b_i(matrix, i) + a_i(matrix, i) * p[i - 1])

def x_i(i, p, q, answer):
    return p[i] * answer[i + 1] + q[i]

def a_i(matrix, i):
    if i == 0:
        return 0
    return matrix[i, i - 1]

def b_i(matrix, i):
    return matrix[i, i]

def c_i(matrix, i):
    if i == (shape[0] - 1):
        return 0
    return matrix[i, i + 1]

def d_i(matrix, i):
    return matrix[i, -1]

In [8]:
print(solve_sweep(matrix_array))

p coefficients
[0.5625, -0.5882352941176471, 0.46788990825688076, 0.424124513618677, -0.0]
q coefficients
[-1.6875, 8.294117647058824, 10.403669724770642, 0.39299610894941606, -8.0]
[ 0.  3.  9. -3. -8.]


## Метод итераций

In [9]:
matrix_array = np.loadtxt('matrix_iterations')
print(matrix_array)
shape = matrix_array.shape

[[ 10.   0.   2.   4. 110.]
 [  2.  16.  -3.   8. 128.]
 [  1.   5.  11.  -4. 102.]
 [  8.   1.   6. -17.  81.]]


In [10]:
def check_diagonal_dominance(matrix):
    shape = matrix.shape
    for row in range (shape[0]):
        if not (row_dominance(matrix, row) or column_dominance(matrix, row)):
            return False

    return True

def row_dominance(matrix, row):
    row_sum = np.sum(np.abs(matrix), axis=1)
    return np.abs(matrix[row, row]) >= (row_sum[row] - np.abs(matrix[row, -1]) - np.abs(matrix[row, row]))

def column_dominance(matrix, column):
    column_sum = np.sum(np.abs(matrix), axis=0)
    return abs(matrix[column, column]) >= column_sum[column] - abs(matrix[column, column])


def solve_iterations(matrix, eps):
    if not check_diagonal_dominance(matrix):
        return
    matrix = get_non_zero_elements_at_diagonal(matrix)

    beta = get_betas_column(matrix)
    alpha = get_alphas_column(matrix)

    x = np.copy(beta)
    iterations = 0
    while True:
        iterations += 1
        x_i = beta + np.matmul(alpha, x)
        if is_accuracy_achieved(x_i, x, eps):
            return x_i, iterations
        x = x_i

def is_accuracy_achieved(x, x_prev, eps):
    return np.max(np.abs(x - x_prev)) <= eps

def get_betas_column(matrix):
    beta = np.zeros((shape[0], 1))
    for i in range(0, shape[0]):
        beta[i, :] = matrix[i, -1] / matrix[i, i]

    return beta

def get_alphas_column(matrix):
    alpha = np.zeros((shape[0], shape[0]))

    for i in range(0, shape[0]):
        for j in range(0, shape[0]):
            if i == j:
                continue
            else:
                alpha[i, j] = -matrix[i, j] / matrix[i, i]

    return alpha

def get_non_zero_elements_at_diagonal(matrix):
    for column in range (0, shape[0]):
        if matrix[column, column] == 0:
            set_non_zero_element_for_column(matrix, column)

    return matrix

def set_non_zero_element_for_column(matrix, column):
    for row in range (0, shape[0]):
        if matrix[row, column] != 0:
            matrix[[column, row]] = matrix[[row, column]]
            return

In [11]:
print("answer and iterations")
print(solve_iterations(matrix_array, 0.00001))

answer and iterations
(array([[9.],
       [7.],
       [6.],
       [2.]]), 22)


## Метод Зейделя

In [12]:
def solve_zeidel(matrix, eps):
    if not check_diagonal_dominance(matrix):
        return

    matrix = get_non_zero_elements_at_diagonal(matrix)

    beta = get_betas_column(matrix)
    alpha = get_alphas_column(matrix)

    x = beta
    iterations = 0

    while True:
        iterations += 1
        x_n = np.copy(x)
        for i in range (shape[0]):
             x_n[i] = beta[i] + sum(alpha[i, j] * x_n[j] for j in range(i)) + sum(alpha[i, j] * x[j] for j in range(i, shape[0]))

        if is_accuracy_achieved(x_n, x, eps):
            return x_n, iterations

        x = x_n



In [13]:
print("answer and iterations")
print(solve_zeidel(matrix_array, 0.00001))

answer and iterations
(array([[9.],
       [7.],
       [6.],
       [2.]]), 12)


## Метод вращений

In [14]:
matrix_array = np.loadtxt('matrix_rotations')
print(matrix_array)
shape = matrix_array.shape

[[4. 2. 1.]
 [2. 5. 3.]
 [1. 3. 6.]]


In [15]:
def solve_rotations(matrix, eps):
    if not check_symmetry(matrix):
        return

    a = np.copy(matrix)
    rotation_matrices = []
    iterations = 0
    while True:
        i, j = get_position_of_max_non_diagonal_element(a)
        max_element = a[i, j]
        phi = 1/2 * np.arctan(2*max_element / (a[i, i] - a[j, j]))
        rotation_matrix = build_rotation_matrix((i, j), shape[0], phi)
        a_k = np.matmul(np.matmul(rotation_matrix.T, a), rotation_matrix)
        if is_accuracy_achieved(a_k, eps):
            return np.diagonal(a_k), get_eigenvectors(rotation_matrices), iterations
        else:
            a = a_k
            rotation_matrices.append(rotation_matrix)
            iterations += 1

def check_symmetry(matrix):
    return np.allclose(matrix, matrix.T)


def get_position_of_max_non_diagonal_element(matrix):
    matrix_for_subtract = np.identity(shape[0]) * np.diagonal(matrix)
    matrix = np.abs(matrix - matrix_for_subtract)
    element =  np.max(matrix)
    indexes =  np.where(matrix == element)
    return indexes[0][0], indexes[1][0]

def build_rotation_matrix(position_of_max_element, size, phi):
    rotation_matrix = np.identity(size)
    i, j = position_of_max_element
    rotation_matrix[i, j] = -np.sin(phi)
    rotation_matrix[j, i] = np.sin(phi)
    rotation_matrix[i, i] = rotation_matrix[j, j] = np.cos(phi)
    return rotation_matrix

def is_accuracy_achieved(matrix, eps):
    matrix_for_subtract = np.identity(shape[0]) * np.diagonal(matrix)
    matrix = np.power(matrix - matrix_for_subtract, 2)
    sum_of_elements = np.power(np.sum(matrix) / 2, 0.5)
    return sum_of_elements < eps

def get_eigenvectors(rotation_matrices):
    result = rotation_matrices[0]
    if len(rotation_matrices) > 1:
        for i in range(1, len(rotation_matrices)):
            result = np.matmul(result, rotation_matrices[i])

    return result

In [16]:
eigenvalues, eigenvectors, iterations = solve_rotations(matrix_array, 0.00001)
print("iterations")
print(iterations)
print("eigenvalues")
print(eigenvalues)
print("eigenvectors")
for vector in eigenvectors.T:
    print(vector, end=",\n")

iterations
6
eigenvalues
[3.73  1.921 9.348]
eigenvectors
[ 0.776  0.195 -0.6  ],
[-0.515  0.746 -0.423],
[0.365 0.637 0.679],


## QR-разложение

In [17]:
matrix_array = np.loadtxt('matrix_qr')
print(matrix_array)
shape = matrix_array.shape

[[ 5. -1. -2.]
 [-4.  3. -3.]
 [-2. -1.  1.]]


In [18]:
def solve_qr(matrix, eps):
    a = matrix
    iterations = 0
    pQ = np.eye(matrix.shape[0])
    while True:
        iterations += 1
        h_matrices = []
        r = a
        for column in range(shape[1] - 1):
            v = get_v_vector(r, column)
            h = get_householder_matrix(v)
            h_matrices.append(h)
            r = np.matmul(h, r)

        q = multiply_matrices(h_matrices)
        a = np.matmul(r, q)
        pQ = np.matmul(pQ, q)
        if is_accuracy_achieved(a, eps):
            print("iterations:")
            print(iterations)
            return np.diagonal(a), pQ

def get_v_vector(matrix, column):
    v = np.zeros(shape[0])
    v[column] =  matrix[column, column] + np.sign(matrix[column, column]) * np.sqrt(np.sum(np.power(matrix[column:, column], 2)))
    v[column + 1:] = matrix[column + 1:, column]
    return np.atleast_2d(v).T

def get_householder_matrix(v):
    e = np.identity(v.shape[0])
    return e - 2 * np.matmul(v, v.T)/np.matmul(v.T, v)

def multiply_matrices(matrices):
    result = matrices[0]
    if len(matrices) > 1:
        for i in range(1, len(matrices)):
            result = np.matmul(result, matrices[i])

    return result

def multiply_reversed(matrices):
    result = matrices[-1]
    if len(matrices) > 1:
        for i in reversed(range(0, len(matrices) - 1)):
            result = np.matmul(result, matrices[i])

    return result

def is_accuracy_achieved(matrix, eps):
    sum_of_subdiagonal_elements = 0
    for i in range(1, shape[0]):
        sum_of_subdiagonal_elements += np.sum(np.power(np.diagonal(matrix, -i), 2))
    return np.sqrt(sum_of_subdiagonal_elements) < eps

In [19]:
print("eigenvalues")
values, vectors = solve_qr(matrix_array, 0.00001)
print(values)
for vector in vectors.T:
    print(vector, end=",\n")

eigenvalues
iterations:
23
[ 6.384  3.839 -1.224]
[-0.694  0.709  0.126],
[-0.55  -0.636  0.542],
[0.464 0.307 0.831],
